In [1]:
# !pip install deepface
# !pip install opencv-python

In [1]:
import cv2


In [4]:
alg = "haarcascade_frontalface_default.xml"
haar_cascade = cv2.CascadeClassifier(alg)
file_name = "data/image.jpg"

img = cv2.imread(file_name, 0)
grey_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

faces = haar_cascade.detectMultiScale(grey_img, scaleFactor=1.05, minNeighbors=3, minSize=(110, 110))


In [5]:
faces.shape

(6, 4)

In [7]:
i = 0

for x,y,w,h in faces:
    cropped_image = img[y:y+h, x: x+w]
    target_file_name = 'stored_faces/' + str(i) + '.jpg'
    cv2.imwrite(target_file_name, cropped_image)
    i = i+1
    
